In [2]:
import torch
import numpy as np
import pandas as pd
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load embeddings
text_chunks_embeddings_df = pd.read_csv("./embeddings/text_chunks_embeddings-20token.csv")

# Convert embedding column back to np.array
text_chunks_embeddings_df["chunk_embedding"] = text_chunks_embeddings_df["chunk_embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

# Convert np.array to torch.tensor
embeddings = torch.tensor(np.array(text_chunks_embeddings_df["chunk_embedding"].tolist()), dtype=torch.float32).to(device)
embeddings[0]

tensor([ 1.6803e-02, -8.5688e-03,  2.7318e-02, -5.6922e-03, -6.5891e-02,
         2.7175e-02, -4.1958e-03,  1.1161e-02,  1.0069e-02, -1.7585e-02,
        -4.4139e-02, -2.6051e-02,  3.0607e-02,  6.9685e-02, -4.0995e-02,
         5.4779e-02, -5.3712e-02, -1.7699e-02,  5.1627e-03,  5.4990e-02,
         1.4213e-02,  2.1292e-02, -1.6230e-02,  2.1104e-02, -3.0262e-03,
        -6.7299e-02,  2.1653e-02,  3.9387e-02, -2.7637e-02, -3.1806e-02,
        -5.0926e-02,  4.1780e-02, -1.2469e-01,  6.5182e-02,  1.8298e-06,
        -1.5614e-02,  1.6449e-02,  2.7332e-02,  3.8913e-03,  2.1397e-02,
         8.5804e-02,  5.0861e-02, -1.3688e-02, -3.2527e-04, -3.7221e-02,
         6.6467e-02, -1.6561e-02, -1.0265e-02, -8.8885e-04,  1.4564e-02,
        -2.7924e-02,  3.2572e-02,  3.6112e-02,  4.2656e-02,  1.5899e-02,
        -1.9914e-02,  5.6539e-03, -4.5028e-02,  7.5522e-02,  3.4398e-04,
         3.7259e-02,  7.6995e-03, -1.2363e-02, -4.6190e-02,  3.6205e-02,
         1.2471e-02,  4.2370e-02, -1.2253e-01, -4.5

In [3]:
from sentence_transformers import util, SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path='all-mpnet-base-v2',
                                      device=device)

c:\Users\vince\anaconda3\envs\cuda_ai_310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
import time
import openai 

openai.base_url = "http://localhost:1234/v1/"
openai.api_key = "NoNeed"

def get_llm_response(prompt, rag_info, max_tokens=500, temperature=0):
    try:
        start_time = time.time()
        response = openai.chat.completions.create(
            model="llama-3.2-1b-instruct",
            messages=[
                { "role": "system", "content": f"""
                Analyze these user reviews to answer: {prompt}
                    Consider:
                    - Common patterns across reviews
                    - Sentiment trends
                    - Specific user pain points
                    - Changes in feedback over time

                Reviews: {rag_info}
                """ },
                {"role": "system", "content": "Answer in paragraph form." },
                { "role": "user", "content": prompt }
            ],
            max_tokens=max_tokens,
            temperature=temperature,
        )

        print(f"... Time taken: {time.time() - start_time}")
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

In [5]:
prompt = input("Prompt: ")

In [6]:
query_embedding = embedding_model.encode(prompt, convert_to_tensor=True)

from time import perf_counter as pc

start_time = pc()
dot_scores = util.dot_score(a=query_embedding, b=embeddings)[0]
end_time = pc()

print(f"Time taken: {end_time - start_time} seconds")

# Get top 20 chunks
top_20_indices = torch.topk(dot_scores, k=20).indices

Time taken: 0.03446799999983341 seconds


In [7]:
rag_info = ". ".join(text_chunks_embeddings_df.iloc[top_20_indices]["sentence_chunk"])

In [8]:
print(f"Response: {get_llm_response(prompt, rag_info)}")

... Time taken: 27.147341012954712
Response: The user reviews provide a comprehensive analysis of the issues they have encountered with the Instagram app, including bugs, glitches, and frustrating experiences. Here are some key points to consider:

**Common patterns across reviews**: Many users mention that their account was disabled without reason, forcing them to take a "real person test" or undergo additional verification steps. Some users also express frustration with the app's inability to handle certain features, such as posting stories or sharing direct messages.

**Sentiment trends**: The overall sentiment of the reviews is overwhelmingly negative, with many users expressing disappointment and frustration with the app's performance and user experience. However, some users do mention that they are willing to continue using the app if it is fixed and improved.

**Specific user pain points**: Some users highlight specific issues, such as:

* Difficulty testing the app due to lack 